In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pandarallel import pandarallel

In [ ]:
d = pd.read_csv('datasets/taxi_api.csv')
df = pd.DataFrame(pd.to_datetime(d.tpep_pickup_datetime))

In [ ]:
def getDay(param):
    return str(param.strftime("%d"))

def getHour(param):
    return str(param.strftime("%H"))

In [ ]:
pandarallel.initialize()
mask_dia = df.tpep_pickup_datetime.parallel_apply(getDay)
mask_hora = df.tpep_pickup_datetime.parallel_apply(getHour)

In [ ]:
# mask dia
df['dia_hora'] = mask_dia
df.dia_hora = df.dia_hora.astype('str')

In [ ]:
#mask_hora
df['hora'] = mask_hora
df['dia'] = mask_dia
df.hora = df.hora.astype('int')

#horas_dia
df.dia_hora = df.dia_hora + '-' + df.hora.astype('str')

In [ ]:
d = df.groupby(['dia_hora','hora']).size()

In [ ]:
statistics = pd.DataFrame(d.values, columns=['viajes_amount'])

In [ ]:
# get index from statistics
horas = np.zeros((d.values.shape[0],1))
for idx, i in enumerate(d.index):
    horas[idx] = i[1]

statistics['hora'] = horas

In [ ]:
days = statistics.groupby('hora').median().index
y0 = statistics.groupby('hora').median().viajes_amount

In [ ]:
fig = go.Figure([go.Bar(x=days, y = y0)])
fig.show()